In [1]:
import tensorflow.keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, GlobalMaxPool1D, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os 

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
filename = '../dataset_text/miditokens_waitFix.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
miditokens_tempo_and_sig = [tokens.strip().split(' ') for tokens in miditokens]

miditokens = []
for song in miditokens_tempo_and_sig:
    sig = song[2]
    if sig in ['timesig:4/4', 'timesig:3/4', 'timesig:2/4', 'timesig:6/8']:
        miditokens.append(song)
print("Number of songs: {0}".format(len(miditokens)))

Number of songs: 550


In [4]:
tokenizer = Tokenizer(oov_token='x') # token -> int
tokenizer.fit_on_texts(miditokens)

In [5]:
#sample = miditokens[1]
#print(sample)
#print(tokenizer.texts_to_sequences([sample])[0]) # Example of token input -> int output
#print("Sõna sagedus kogu andmestikus", tokenizer.word_counts['wait:0.0'])
#print("Sõna -> indeks teisendus", tokenizer.word_index['wait:0.0'])
#print("Indeks -> sõna teisendus", tokenizer.index_word[55])
print(len(tokenizer.word_index))

310


In [6]:
# Turn all tokens to ints
midiTokensAsInt = tokenizer.texts_to_sequences(miditokens)
#midiTokensAsInt = np.array(midiTokensAsInt)

# Generator variant

In [720]:
SEQ_LEN = 50

with open("../dataset_text/seq2note_int.txt", "a") as f:
    for song in midiTokensAsInt:
        for i in range(0, len(song) - SEQ_LEN, 1):
            song = [str(token) for token in song]
            f.write(' '.join(song[i:i + SEQ_LEN]) + ", " + (song[i + SEQ_LEN]) + '\n')

In [7]:
# https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366

SEQ_LEN = 50
LINES = 0
for song in midiTokensAsInt:
    for idx in range(0, len(song) - SEQ_LEN, 1):
        LINES += 1
print(LINES)

BATCH_SIZE = 512
VOCAB_SIZE = len(tokenizer.word_index)
steps = LINES // BATCH_SIZE

with open("../dataset_text/seq2note_int.txt", "r") as f:
    lines = f.readlines()

def batchGenerator(trainData, lines, steps, VOCAB_SIZE, LINES, SEQ_LEN=50, BATCH_SIZE=32):
    lastLine = 0
    while True:
        
        # https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
        
        X_train = []
        y_train = []
        
        for idx in range(lastLine, min(lastLine + BATCH_SIZE, LINES), 1):
            sample = lines[idx].split(", ")
            X_train.append([int(i) for i in sample[0].split(" ")])
            y_train.append(int(sample[1]))

        X_train = to_categorical(X_train, num_classes=VOCAB_SIZE+1)
        y_train = to_categorical(y_train, num_classes=VOCAB_SIZE+1)
        
        yield (X_train, y_train)
        
        lastLine += BATCH_SIZE
        if lastLine > LINES:
            lastLine = 0

batchGen = batchGenerator(midiTokensAsInt, lines, steps, VOCAB_SIZE, LINES, SEQ_LEN, BATCH_SIZE)

1357582


In [702]:
#test = next(batchGen)

In [728]:
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(SEQ_LEN, VOCAB_SIZE+1),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE+1))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
#filepath = "../weights/weights-generator/weights-{epoch:02d}-{loss:.4f}.hdf5"    
#checkpoint = ModelCheckpoint(
#    filepath, monitor='loss', 
#    verbose=0,        
#    save_best_only=True,        
#    mode='min'
#)    
#callbacks_list = [checkpoint]     
##model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

model.fit(batchGen, 
          workers=0,
          steps_per_epoch = steps,
          epochs=200,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/200
2651/2651 [==============================] - 392s 148ms/step - loss: 0.8305
Epoch 2/200
2651/2651 [==============================] - 392s 148ms/step - loss: 0.8297
Epoch 3/200
2651/2651 [==============================] - 393s 148ms/step - loss: 0.8276
Epoch 4/200
2651/2651 [==============================] - 393s 148ms/step - loss: 0.8261
Epoch 5/200
2651/2651 [==============================] - 391s 147ms/step - loss: 0.8255
Epoch 6/200
2651/2651 [==============================] - 392s 148ms/step - loss: 0.8254
Epoch 7/200
2651/2651 [==============================] - 391s 148ms/step - loss: 0.8240
Epoch 8/200
2651/2651 [==============================] - 392s 148ms/step - loss: 0.8231
Epoch 9/200
2651/2651 [==============================] - 393s 148ms/step - loss: 0.8211
Epoch 10/200
2651/2651 [==============================] - 391s 148ms/step - loss: 0.8219
Epoch 11/200
2651/2651 [==============================] - 393s 148ms/step - loss: 0.8192
Epoch 12/200
2651/2651 [======

KeyboardInterrupt: 

In [729]:
model.save("LSTM256-generator2.h5")

In [8]:
model = tf.keras.models.load_model("../trained_models/LSTM256-generator2.h5")

In [ ]:
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(X_train.shape[1], X_train.shape[2]),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
## Load the weights to each node
#model.load_weights('../weights/weights.hdf5')

In [29]:
intToNote = dict(map(reversed, tokenizer.word_index.items()))

songTokens = tokenizer.texts_to_sequences([midi2text(open_midi("../testmidis/sonic.mid"))])[0]
pattern = [songTokens[:50]] 

prediction_output = [intToNote[i] for i in list(pattern[0])] # The starting input will be the beginning of the output 

starting_input = text2midi(prediction_output)
starting_input.show("midi")
starting_input.write('midi', fp='../results/sonic_input.mid')

# Range determines the number of tokens to predict
for note_index in range(2000):
    # Convert to acceptable format for trained model
    prediction_input = to_categorical(pattern, num_classes=VOCAB_SIZE+1)
    
    # Predict next token depending on the previous 50 tokens
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    
    if (note_index % 20 == 0):
        ind = np.argpartition(prediction[0], -3)[-3:]
        index = np.random.choice(ind)
    #print(index, ind, randomindex)
    
    result = intToNote[index]
    
    prediction_output.append(result)
    
    pattern = np.append(pattern, index)
    pattern = [pattern[1:len(pattern)]]
    
#print(prediction_output)

In [30]:
genmidistream = text2midi(prediction_output)
genmidistream.show("midi")
genmidistream.write('midi', fp='../results/sonic_output.mid')

'../results/sonic_output.mid'

In [36]:
if "end" in prediction_output:
    l2 = prediction_output[:prediction_output.index("end")+1]
    genmidistream = text2midi(l2)
    genmidistream.show("midi")

In [7]:
from music21 import *

def open_midi(midi_path):
    mf = converter.parse(midi_path)
    return mf

# Restricts possible velocities to 8 values, keeping the number of unique note events smaller
# Resembles ppp, pp, p, mp, mf, f, ff, fff dynamics 
def vModifier(velocity):
    if (velocity == 0):
        return 0
    
    velocity = min(127, ((velocity // 16) + 1) * 16)
    return velocity

def tModifier(tempo):
    if (tempo == 0):
        return 0
    
    tempo = ((tempo // 10) + 1) * 10
    return tempo

# Check if there are notes which should have ended before given offset
def checkForNoteOffEvent(currentOffset, noteOffEvents):
    notesToEnd = []
    
    for noteOffEvent in noteOffEvents: # for (notename, endingOffset)
        if noteOffEvent[1] <= currentOffset:
            notesToEnd.append(noteOffEvent)
            
    return notesToEnd

def midi2text(midifile):
    previousElementOffset = 0.0
    offsetChanged = False

    tempoRetrieved = False
    timeSigRetrieved = False
    
    currentVelocity = 0

    tokens = []
    noteOffEvents = []

    tokens.append("START")

    for element in midifile.flat.elements:
        #print(type(element))

        currentElementOffset = element.offset

        notesToEnd = checkForNoteOffEvent(currentElementOffset, noteOffEvents)

        if (len(notesToEnd) != 0):
            for noteToEnd in notesToEnd:
                difference = float(noteToEnd[1]) - float(previousElementOffset)
                if (difference > 0.01):
                    tokens.append("wait:" + str(round(difference, 5)))
                    previousElementOffset = noteToEnd[1]
                tokens.append("note:" + str(noteToEnd[0]) + ":OFF")
                noteOffEvents.remove(noteToEnd)

        # If offset has increased and we're looking at new notes, add a wait event before adding the new notes
        if (float(currentElementOffset) > float(previousElementOffset + 0.01) and (isinstance(element, note.Note) or isinstance(element, chord.Chord))):
            offsetChanged = True
            difference = float(currentElementOffset - previousElementOffset)
            tokens.append("wait:" + str(round(difference, 5)))

        if (isinstance(element, tempo.MetronomeMark) and not tempoRetrieved):
            tempoRetrieved = True
            tokens.append("tempo:" + str(tModifier(element.number)))

        if (isinstance(element, meter.TimeSignature) and not timeSigRetrieved):
            timeSigRetrieved = True
            tokens.append("timesig:" + str(element.ratioString))

        if (isinstance(element, note.Note)): # This is a note event, add a token for this note
            if (currentVelocity != vModifier(element.volume.velocity)):
                currentVelocity = vModifier(element.volume.velocity)
                tokens.append("velocity:" + str(currentVelocity))
            tokens.append("note:" + str(element.pitch))
            noteOffEvents.append((str(element.pitch), float(currentElementOffset + element.duration.quarterLength), 5))

        if (isinstance(element, chord.Chord)): # This is a chord event, add a token for each note in chord
            for chordnote in element:
                if (currentVelocity != vModifier(element.volume.velocity)):
                    currentVelocity = vModifier(element.volume.velocity)
                    tokens.append("velocity:" + str(currentVelocity))
                tokens.append("note:" + str(chordnote.pitch))
                noteOffEvents.append((str(chordnote.pitch), float(currentElementOffset + element.duration.quarterLength)))

        if (offsetChanged):
            previousElementOffset = currentElementOffset
            offsetChanged = False

    # Finally make sure that all notes that end after the offset of the last element of mf.flat.elements are given an off event.
    for noteToEnd in noteOffEvents.copy():
        difference = float(noteToEnd[1]) - float(previousElementOffset)
        if (difference > 0.01):
            tokens.append("wait:" + str(round(difference, 5)))
            previousElementOffset = noteToEnd[1]
        tokens.append("note:" + str(noteToEnd[0]) + ":OFF")
        noteOffEvents.remove(noteToEnd)
        
    if (len(noteOffEvents) != 0):
        print("Not all notes have note-off events")

    tokens.append("END")
    return tokens

def text2midi(tokens):
    s = stream.Stream()
    
    currentVelocity = 80

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))
            
        if token.startswith("velocity"):
            currentVelocity = int(splitToken[1])

        if token.startswith("note") and not token.lower().endswith("off"):
            noteDuration = 0
            noteName = splitToken[1]

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.lower().endswith("off")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = currentVelocity
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s